In [10]:
# To Do: Create global config object for things that are not tied to a
#        particular dataset configuration but may still be changeable, e.g.
#        user messages related to input validation. Or maybe not this is an
#        interactive notebook not some compiled software.

import ipywidgets as widgets
import json
import warnings

from IPython.display import display

with open('./default/default_employees.json') as default_configuration_json:
    config = json.load(default_configuration_json)


l3_config_dataset_details = config['overview']['dataset_details']

l3_config_organisation_size   = config['employee_properties']['organisation_size']
l3_config_employment_type_mix = config['employee_properties']['employment_type_mix']

In [11]:
# Generate list of unique IDs to tag procedurally generated widgets with (i.e.
# those we haven't given an explict variable name to) so that we can interact
# with them reliably after creation.

# Using a simple one dimensional list where the index doubles as the id and the
# value is a boolean where True indicates that the id is available for use and
# false indicates that the id is already assigned to a widget.

# If we are over 1000 procedurally generaetd widgets I expect that something
# has gone terribly wrong.
procedurally_generated_widget_ids = [True for i in range(1000)]
current_widget_id = 0
    
def obtain_new_widget_id():
    global current_widget_id
    current_widget_id += 1
    return current_widget_id

def release_widget_id(widget_id):
    procedurally_generated_widget_ids[widget_id] = True

In [12]:
# Standard Properties
l3_box_layout = widgets.Layout(
      display         = 'flex'
    , flex_flow       = 'column'
    , align_content   = 'center'
    , justify_content = 'center'
)

l3_text_layout = widgets.Layout(
    display           = 'flex'
  , justify_content   = 'center'
  , width             = '25%'
)

l3_input_layout = widgets.Layout(
    width             = 'max_content'
)

l3_input_style = {
    'description_width': '50%'
}

In [13]:
# Global Object to hold all widgets based on where they will be rendered. While
# ipywidgets supports re-rendering the same widget in multiple/cells views we
# will not use this methodology in this notebook.
ws = {
      'l1_tab': widgets.Tab()

# Level 2: Overview ----------------------------------------------------------#
    , 'l2_overview':                           widgets.Box()
    , 'l3_dataset_details':                    widgets.Box(layout = l3_box_layout)
    , 'l3_dataset_details_load_configuration': widgets.Button(description = 'Load Configuration')
    , 'l3_dataset_details_save_configuration': widgets.Button(description = 'Save Configuration')

    , 'l3_dataset_details_configuration_name': widgets.Text(
          value       = l3_config_dataset_details['configuration_name']
        , description = 'Configuration Name'
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_configuration_last_modified': widgets.Text(
          value       = l3_config_dataset_details['last_modified']
        , description = 'Last Modified'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )

    , 'l3_dataset_details_dataset_name': widgets.Text(
          value       = l3_config_dataset_details['dataset_name']
        , description = 'Dataset Name'
        , disabled    = True
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
# ----------------------------------------------------------------------------#

# Level 2: Employee Properties ----------------------------------------------#
    , 'l2_employee_properties': widgets.Accordion()
    
    , 'l3_general_organisation_size': widgets.BoundedIntText(
          value       = l3_config_organisation_size['value']
        , min         = 100
        , max         = 100000
        , description = l3_config_organisation_size['description']
        , disabled    = False
        , layout      = l3_input_layout
        , style       = l3_input_style
    )
    
    , 'l3_employment_type_mix':              widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_header':       widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_body':         widgets.Box(layout = l3_box_layout)
    , 'l3_employment_type_mix_user_message': widgets.Label(value = '')
    , 'l3_employment_type_mix_add_type':     widgets.Button(description = 'Add Type')
    
    , 'l3_employment_type_mix_total': widgets.IntText(
          description = 'Total'
        , value       = 0
        , disabled    = True
      # , layout      = l3_input_layout
      # , style       = l3_input_style
    )
    

# ----------------------------------------------------------------------------#
}

In [14]:
# Level 3: Overview / Dataset Details
ws['l3_dataset_details'].children = [
      ws['l3_dataset_details_load_configuration']
    , ws['l3_dataset_details_save_configuration']
    , ws['l3_dataset_details_configuration_name']
    , ws['l3_dataset_details_dataset_configuration_last_modified']
    , ws['l3_dataset_details_dataset_name']
]

In [15]:
# Level 2: Overview
# Maybe down the track we will want additional information in level 2 overview.
ws['l2_overview'] = ws['l3_dataset_details']

In [19]:
# Level 3: Employee Properties / Employment Type Mix
# To Do:  Create functionality for users to add/remove/modify employment types descriptions.
#         Find a better way to address the input values rather than assuming that index 1 of each HBox row will always be the value.
#         Lament the fact that we can't pass arguments to button handlers which makes it trickier to genericise functions for dynamically creating/removing children.
def l3_employment_type_mix_on_value_change(change):
  if change['old'] != change['new']:
    ws['l3_employment_type_mix_total'].value += change['new'] - change['old']

  if ws['l3_employment_type_mix_total'].value == 100:
    ws['l3_employment_type_mix_user_message'].value = 'Employment types total 100% Good to go!'
  elif ws['l3_employment_type_mix_total'].value > 100:
    ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total over 100% Please correct mix.'
  else:
    ws['l3_employment_type_mix_user_message'].value = 'Warning: Employment types total under 100% Please correct mix.'

def l3_employment_type_mix_add(additional_mix_details_list = None):
    if additional_mix_details_list == None or additional_mix_details_list.__class__.__name__ == 'Button':
        additional_mix_details_list = [{'mix_name': 'enter employment type', 'mix_percentage': 0}]

    additional_mix_list = []

    for index, input in enumerate(additional_mix_details_list):
        swap_widget_id = str(obtain_new_widget_id())
        
        additional_mix_list.append(widgets.HBox(children = [
              widgets.Text(value = input['mix_name'])
            , widgets.BoundedIntText(
                value       = input['mix_percentage'] * 100
              , min         = 0
              , max         = 100
              , step        = 1
              , disabled    = False
              , layout      = l3_input_layout
              , style       = l3_input_style
            )
            , widgets.Button(
                description = 'Delete'
              , tooltip = 'delete ' + input['mix_name'] + ' | ID=' + swap_widget_id + ' | Parent=l3_employment_type_mix_body'
            )
        ]))
        
    for input in additional_mix_list:
      input.children[1].observe(handler = l3_employment_type_mix_on_value_change, names = 'value')
      input.children[2].on_click(l3_employment_type_mix_delete)

    ws['l3_employment_type_mix_body'].children += tuple(additional_mix_list)


def l3_employment_type_mix_delete(e):
    for index, input in enumerate(ws['l3_employment_type_mix_body'].children):
        if e.tooltip == input.children[2].tooltip:
            input.children[1].value = 0
            l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})
            input.children[0].close()
            input.children[1].close()
            input.children[2].close()
            
            # If we close out the container attempts to add new rows fail with error:
            # AttributeError: 'NoneType' object has no attribute 'comm_id'
            # Appears to be triggered when trying to extend ws['l3_employment_type_mix_body'].children
            # for the time being we'll 0 out the values an leave the containter
            # box in an 'open' state.
            # input.close()
            break


def l3_employment_type_mix_values_to_json():
  dumps({input.description: input.value / 100 for input in ws['l3_employment_type_mix_body'].children})


ws['l3_employment_type_mix_add_type'].on_click(l3_employment_type_mix_add)

ws['l3_employment_type_mix_header'].children = [
      ws['l3_employment_type_mix_user_message']
    , ws['l3_employment_type_mix_add_type']
    , ws['l3_employment_type_mix_total']
]

# Generate list of employment mixes based on current configuration.
l3_employment_type_mix_add(l3_config_employment_type_mix['values'])

ws['l3_employment_type_mix_total'].value = sum([input.children[1].value for input in ws['l3_employment_type_mix_body'].children])
l3_employment_type_mix_on_value_change({'old': 0, 'new': 0})

In [20]:
# Level 2: Employee Properties
ws['l2_employee_properties'].children = [ws['l3_general_organisation_size'], ws['l3_employment_type_mix']]
ws['l2_employee_properties'].titles   = ['Organisation Size', 'Employment Type Mix (percentage)']

In [18]:
ws['l1_tab'].children = [
      ws['l2_overview']
    , ws['l2_employee_properties']
    , ws['l2_employee_properties']
    , ws['l2_employee_properties']
]

ws['l1_tab'].titles = [
      'Overview'
    , 'Employee Properties'
    , 'Divisions'
    , 'Teams'
  ]
  
display(ws['l1_tab'])